In [60]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from matplotlib import pyplot as plt
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Functions.original.utils.undo_dummy import back_from_dummies
import joblib

import seaborn as sns
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
import itertools
import plotly.express as px 
from scipy import stats
from copy import deepcopy
from sklearn.model_selection import GroupShuffleSplit
from sklearn.datasets import fetch_openml
warnings.filterwarnings('ignore')

from Functions import dataprep as prep
from Functions import metrics
from sklearn.datasets import fetch_openml
import pandas as pd
from _1_DataPrep import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
train = pd.read_pickle('./data/common_dataprep/train_with_ei.pickle')
test = pd.read_pickle('./data/common_dataprep/test_with_ei.pickle')

In [62]:
def make_glms(train, test, base_formula = ''):
    # No Input
    formula0 = "ClaimNb ~ 1"
    formula1 = f"ClaimNb ~ {base_formula}  VehPowerGLM + C(VehAgeGLM, Treatment(reference=2)) + C(DrivAgeGLM, Treatment(reference=5)) + BonusMalusGLM + VehGas + DensityGLM + C(Region, Treatment(reference='R24'))"
    formula2 = f"{formula1} + VehBrand"
    formula3 = f"{formula2} + AreaGLM"

    formulas = [formula0, formula1, formula2, formula3]
    devs = []
    
    for formulaa in formulas:
        glm0 = smf.glm(formula=formulaa, data=train, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train['Exposure'])).fit()
        devs += [metrics.poisson_deviance(glm0.predict(test, offset=np.log(test['Exposure'])), test['ClaimNb'])]
    
    assert min(devs) != devs[0], f'Issue with metrics: {dev}; random model predicts best?!'
    return min(devs)
    

In [ ]:
train['VehPowerGLM']

In [ ]:
print(
    
    make_glms(train, test),
    make_glms(train, test, base_formula = 'EI_Density + EI_DrivAge + EI_BonusMalus1 + EI_BonusMalus2 + EI_VehAge +'),
    make_glms(train, test, base_formula = 'GDV_Area + GDV_VehAge + GDV_DrivAge + '),
    make_glms(train, test, base_formula = 'GDV_Area + GDV_VehAge + GDV_DrivAge + EI_Density + EI_DrivAge + EI_BonusMalus1 + EI_BonusMalus2 + EI_VehAge +')
)

In [ ]:
train_gan = pd.read_pickle('./data/gan_generated/sample_no_ei.pickle')
ss = joblib.load('data/gan_dataprep/scaler.pickle')
dftemp = pd.read_pickle('data/gan_dataprep/train_gan.pickle')
train_gan = pd.DataFrame(ss.inverse_transform(train_gan), columns= dftemp.columns)
train_gan = back_from_dummies(train_gan)
train_gan.head()

In [ ]:
train_gan = prepare_gandata_for_regression(train_gan)

In [ ]:
cols = ['ClaimNb', 'VehBrand', 'VehGas', 'Region', 'Area', 'VehPower', 'VehAge',
       'DrivAge', 'Density', 'BonusMalus', 'Exposure', 'DensityGLM',
       'BonusMalusGLM', 'AreaGLM', 'VehPowerGLM', 'VehAgeGLM', 'DrivAgeGLM']
joined = pd.concat([train_gan, train])[cols]

In [ ]:
# No Input
formula0 = "ClaimNb ~ 1"
formula1 = f"ClaimNb ~   VehPowerGLM + C(VehAgeGLM, Treatment(reference=2)) + C(DrivAgeGLM, Treatment(reference=5)) + BonusMalusGLM + VehGas + DensityGLM + C(Region, Treatment(reference='R24'))"
formula2 = f"{formula1} + VehBrand"
formula3 = f"{formula2} + AreaGLM"

formulas = [formula0, formula1, formula2, formula3]
devs = []

for formulaa in formulas:
    glm0 = smf.glm(formula=formulaa, data=train_gan, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train_gan['Exposure'])).fit()
    devs += [metrics.poisson_deviance(glm0.predict(test, offset=np.log(test['Exposure'])), test['ClaimNb'])]
    print(devs)

In [67]:
cols = ['ClaimNb', 'VehBrand', 'VehGas', 'Region', 'Area', 'VehPower', 'VehAge',
       'DrivAge', 'Density', 'BonusMalus', 'Exposure', 'DensityGLM',
       'BonusMalusGLM', 'AreaGLM', 'VehPowerGLM', 'VehAgeGLM', 'DrivAgeGLM']
joined = pd.concat([train_gan, train])[cols]

In [ ]:
# No Input
formula0 = "ClaimNb ~ 1"
formula1 = f"ClaimNb ~   VehPowerGLM + C(VehAgeGLM, Treatment(reference=2)) + C(DrivAgeGLM, Treatment(reference=5)) + BonusMalusGLM + VehGas + DensityGLM + C(Region, Treatment(reference='R24'))"
formula2 = f"{formula1} + VehBrand"
formula3 = f"{formula2} + AreaGLM"

formulas = [formula0, formula1, formula2, formula3]
devs = []

for formulaa in formulas:
    glm0 = smf.glm(formula=formulaa, data=train_gan, family=sm.families.Poisson(link=sm.families.links.log()), offset=np.log(train_gan['Exposure'])).fit()
    devs += [metrics.poisson_deviance(glm0.predict(test, offset=np.log(test['Exposure'])), test['ClaimNb'])]
    print(devs)

[32.87212129298799]


In [ ]:
Bootstrap(9, n_bootstraps=3, n_train=5, n_test=4, random_state=0)